In [5]:
#Importing pandas and loading sheets of interest into pandas dataframes
import pandas as pd

path = "../"
Main_df = pd.read_excel(path + "Premier League Promotion Analysis.xlsx")
Survived = pd.read_excel(path + "Premier League Promotion Analysis.xlsx", sheet_name= "Season Summary")

        Season           Team  Total_Matches  Total_Wins  Total_Draws  \
0    1993-1994        Arsenal             42          18           17   
1    1993-1994    Aston Villa             42          15           12   
2    1993-1994      Blackburn             42          25            9   
3    1993-1994        Chelsea             42          13           12   
4    1993-1994       Coventry             42          14           14   
..         ...            ...            ...         ...          ...   
642  2024-2025  Nott'm Forest             23          13            5   
643  2024-2025    Southampton             23           1            3   
644  2024-2025      Tottenham             23           7            3   
645  2024-2025       West Ham             23           7            6   
646  2024-2025         Wolves             23           4            4   

     Total_Losses  Total_Goals_For  Total_Goals_Against  Goal_Difference  \
0               7               53             

In [15]:
#Initialising a dictionary to store lists of relegated and survived teams for each season
season_stats = {}

#Looping through each season
for season in Survived['Season'].unique():
    
    #Filtering the dataframe for the current season
    season_df = Survived[Survived['Season'] == season]
    
    #Initialising lists for relegated and survived teams
    relegated_teams = []
    survived_teams = []
    relegated_after_promotion_teams = []
    
    #Looping through the rows in the filtered dataframe
    for index, row in season_df.iterrows():
        if row['End_Status'] == 'Relegated':
            relegated_teams.append(row['Team'])  
        elif row['End_Status'] == 'Survived':
            survived_teams.append(row['Team'])
        elif row['End_Status'] == 'Relegated After Promotion':
            relegated_after_promotion_teams.append(row['Team'])
    
    #Storing the lists in the dictionary
    season_stats[season] = {
        'Relegated': relegated_teams,
        'Survived': survived_teams,
        'Relegated_After_Promotion': relegated_after_promotion_teams
    }

In [25]:
team_stats_df = Survived
match_stats_df = Main_df

#Initialising a dictionary to store average shots on target for each group per season
average_shots_stats = {}

#Looping through each season in season_stats
for season, groups in season_stats.items():
    
    #Filtering match statistics for the current season
    season_match_stats = match_stats_df[match_stats_df['Season'] == season]
    
    #Initialising a dictionary to store average shots on target for each group
    group_avg_shots = {}

    #Looping through the three groups: "Survived", "Relegated", "Relegated_After_Promotion"
    for group, teams in groups.items():
        #Initialising a list to store shots on target for this group
        shots_on_target = []

        #Looping through the teams in the current group
        for team in teams:
            #Filtering match statistics where the team is either the home or away team
            home_matches = season_match_stats[season_match_stats['HomeTeam'] == team]
            away_matches = season_match_stats[season_match_stats['AwayTeam'] == team]

            #Extracting the "HST" (Home Shots on Target) for home matches
            shots_on_target.extend(home_matches['HST'])

            #Extracting the "AST" (Away Shots on Target) for away matches
            shots_on_target.extend(away_matches['AST'])

        #Calculating the average shots on target for the current group
        avg_shots_on_target = pd.Series(shots_on_target).mean()

        #Storing the result in the group_avg_shots dictionary
        group_avg_shots[group] = avg_shots_on_target

    #Storing the average shots for the current season in the main dictionary
    average_shots_stats[season] = group_avg_shots

         Date        HomeTeam        AwayTeam  FTHG  FTAG FTR  HTHG  HTAG  \
0    14/08/93         Arsenal        Coventry     0     3   A   NaN   NaN   
1    14/08/93     Aston Villa             QPR     4     1   H   NaN   NaN   
2    14/08/93         Chelsea       Blackburn     1     2   A   NaN   NaN   
3    14/08/93       Liverpool  Sheffield Weds     2     0   H   NaN   NaN   
4    14/08/93        Man City           Leeds     1     1   D   NaN   NaN   
..        ...             ...             ...   ...   ...  ..   ...   ...   
457  07/05/94  Sheffield Weds        Man City     1     1   D   NaN   NaN   
458  07/05/94         Swindon           Leeds     0     5   A   NaN   NaN   
459  07/05/94       Tottenham             QPR     1     2   A   NaN   NaN   
460  07/05/94        West Ham     Southampton     3     3   D   NaN   NaN   
461  08/05/94      Man United        Coventry     0     0   D   NaN   NaN   

     HTR Referee  ...  AF  HC  AC  HY  AY  HR  AR     Season  HomePoints  \

In [31]:
#Saving the results to an Excel file
output_file = "average_shots_on_target_stats.xlsx"
output_df = pd.DataFrame(average_shots_stats).transpose()
output_df.to_excel(output_file)

print(f"Average shots on target stats have been saved to {output_file}")

Average shots on target stats have been saved to average_shots_on_target_stats.xlsx
